## Part 1: Importing the libraries and dataset

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [6]:
tf.__version__

'2.2.0'

In [67]:
data = pd.read_csv('Churn_Modelling.csv')

In [68]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [69]:
X = data.iloc[:, 3:-1].values #excluding row number, customer id and surname
y = data['Exited'].values

In [70]:
print(data.shape,X.shape,y.shape)

(10000, 14) (10000, 10) (10000,)


In [71]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [72]:
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

## Part 2: Data Preprocessing

In [73]:
data.isna().sum() #no missing values

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [74]:
X[:,2]

array(['Female', 'Female', 'Female', ..., 'Female', 'Male', 'Female'],
      dtype=object)

In [75]:
#Encoding gender column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [76]:
X[:,2]

array([0, 0, 0, ..., 0, 1, 0], dtype=object)

In [77]:
X[:,1]

array(['France', 'Spain', 'France', ..., 'France', 'Germany', 'France'],
      dtype=object)

In [78]:
#One hot encoding Geography column 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])],remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [79]:
X #first column for France, second column for Germany and 3rd column for Spain

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

In [80]:
#splitting the dataset into training and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [81]:
#feature scaling (mandatory for neural net training)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test) #using only transform to avoid any data leakages

## Part 3: Building the ANN

In [35]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [36]:
# Initializing the ANN
ann = Sequential()

# Adding the input layer and the first hidden layer
ann.add(Dense(units = 6, activation = 'relu'))

# Adding the second hidden layer
ann.add(Dense(units = 6, activation = 'relu'))

# Adding the output layer
ann.add(Dense(units = 1, activation = 'sigmoid')) # one layer in cases of binary classification

## Part 3: Training the ANN

In [46]:
#Compiling the ANN
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'] ) #adam performs stochastic gradient descent

In [49]:
ann.fit(x = X_train, y = y_train, batch_size = 32, epochs = 100, validation_data = (X_test, y_test))

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.6165 - accuracy: 0.6885 - val_loss: 0.5171 - val_accuracy: 0.7970
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4969 - accuracy: 0.7955 - val_loss: 0.4704 - val_accuracy: 0.7975
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4597 - accuracy: 0.7993 - val_loss: 0.4353 - val_accuracy: 0.8085
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4299 - accuracy: 0.8129 - val_loss: 0.4120 - val_accuracy: 0.8235
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4119 - accuracy: 0.8248 - val_loss: 0.3966 - val_accuracy: 0.8370
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3980 - accuracy: 0.8336 - val_loss: 0.3819 - val_accuracy: 0.8460
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3853 - accuracy: 0.8399 - val_loss: 0.3706 - val_accuracy: 0.8555

## Part 4: Making the predictions and evaluating the model

In [50]:
data.head(1)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,101348.88,1


#### Predicting for a single new observation

In [110]:
#input data
test = np.array([[600,'France','Male',40,3,60000,2,1,1,50000]])

#preprocessing
test[:,2] = le.fit_transform(test[:,2]) #encoding gender column
test = np.array(ct.transform(test)) #encoding geography column
test = sc.transform(test) #feature scaling

#predicting the output class
print('Probability',ann.predict(test))
print('Outcome',ann.predict(test)>0.5)

Probability [[0.05041772]]
Outcome [[False]]


#### Predicting the test set results

In [113]:
y_pred = ann.predict_classes(X_test)

#### Making the classification report

In [115]:
from sklearn.metrics import classification_report

In [116]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1595
           1       0.75      0.49      0.59       405

    accuracy                           0.86      2000
   macro avg       0.81      0.73      0.76      2000
weighted avg       0.85      0.86      0.85      2000



**Note:** This code file was reproduced as a follow-along of the ML course - https://www.udemy.com/course/machinelearning/